Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots,ApproxFun

In [2]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,[0,T])
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

Φ = A\eye(n);

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

2.220446049250313e-16 - 0.2683546905319052im
6.10405823109339e-17 + 0.268354690531905im
0.037475319732289236 + 0.9999999999999999im
-0.03747531973228904 + 0.9999999999999997im

Floquet Multipliers = 

0.6651802570066612 - 0.7466828146465897im
0.6651802570066613 + 0.7466828146465889im
-1.1249427991479368 + 6.373407186201634e-16im
-0.888934086921956 + 8.983951009601114e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,[0,10T])
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.4296,96.9128 43.4342,96.8907 43.4434,96.8463 43.4572,96.78 43.4756,96.6916 43.4986,96.5815 43.5262,96.4498 43.5584,96.2968 43.5951,96.1227 43.6365,95.9279 
 43.6825,95.7126 43.7331,95.4774 43.7883,95.2226 43.848,94.9487 43.9124,94.6563 43.9813,94.3459 44.0549,94.018 44.133,93.6734 44.2157,93.3128 44.303,92.9367 
 44.3949,92.5461 44.4914,92.1416 44.5925,91.7242 44.6981,91.2947 44.8083,90.854 44.9231,90.4031 45.0425,89.9429 45.1664,89.4746 45.2949,88.9992 45.428,88.5177 
 45.5657,88.0314 45.7079,87.5414 45.8546,87.0489 46.006,86.5552 46.1619,86.0616 46.3223,85.5693 46.4873,85.0798 46.6568,84.5943 46.8309,84.1143 47.0096,83.6412 
 47.1927,83.1764 47.3804,82.7215 47.5727,82.2779 47.7694,81.847 47.9707,81.4306 48.1766,81.03 48.3869,80.6468 48.6018,80.2825 48.8211,79.9388 49.045,79.6172 
 49.2734,79.3191 49.5063,79.0462 49.7437,78.8 49.9856,78.582 50.232,78.3937 50.4828,78.2366 50.7382,78.1121 50.998,78.0216 51.2623,77.9666 51.5311,77.9483 
 51.8043,77.9681 52.082,78.0272 52.3641,78.1268 52.6507,78.268 52.9418,78.4519 53.2373,78.6795 53.5372,78.9517 53.8415,79.2693 54.1503,79.6331 54.4635,80.0439 
 54.7811,80.502 55.1032,81.0081 55.4296,81.5625 55.7604,82.1655 56.0957,82.8173 56.4353,83.5178 56.7793,84.267 57.1277,85.0647 57.4804,85.9107 57.8375,86.8045 
 58.199,87.7455 58.5648,88.7331 58.935,89.7664 59.3095,90.8445 59.6884,91.9663 60.0716,93.1307 60.4591,94.3364 60.8509,95.5818 61.247,96.8655 61.6474,98.1858 
 62.0522,99.5408 62.4612,100.929 62.8745,102.348 63.2921,103.795 63.7139,105.269 64.14,106.768 64.5704,108.289 65.005,109.829 65.4438,111.386 65.8869,112.958 
 66.3342,114.542 66.7858,116.135 67.2415,117.736 67.7015,119.34 68.1656,120.947 68.6339,122.552 69.1064,124.154 69.5831,125.75 70.064,127.338 70.549,128.915 
 71.0381,130.48 71.5314,132.029 72.0288,133.561 72.5304,135.074 73.036,136.567 73.5458,138.038 74.0596,139.484 74.5776,140.907 75.0996,142.303 75.6257,143.673 
 76.1559,145.016 76.6901,146.332 77.2284,147.621 77.7707,148.884 78.3171,150.12 78.8674,151.33 79.4218,152.517 79.9801,153.681 80.5425,154.825 81.1088,155.95 
 81.6791,157.059 82.2534,158.155 82.8316,159.24 83.4138,160.32 83.9999,161.396 84.59,162.474 85.1839,163.557 85.7817,164.651 86.3835,165.76 86.9891,166.89 
 87.5986,168.045 88.212,169.232 88.8292,170.456 89.4503,171.723 90.0752,173.04 90.7039,174.411 91.3364,175.843 91.9728,177.341 92.6129,178.913 93.2568,180.564 
 93.9045,182.298 94.5559,184.122 95.2111,186.041 95.87,188.059 96.5327,190.18 97.1991,192.409 97.8691,194.748 98.5429,197.201 99.2203,199.769 99.9015,202.454 
 100.586,205.257 101.275,208.178 101.967,211.215 102.662,214.368 103.362,217.634 104.065,221.008 104.771,224.488 105.481,228.067 106.195,231.74 106.912,235.499 
 107.633,239.337 108.357,243.244 109.085,247.211 109.816,251.226 110.551,255.279 111.289,259.356 112.031,263.445 112.776,267.53 113.525,271.598 114.277,275.633 
 115.032,279.619 115.791,283.539 116.554,287.376 117.319,291.113 118.088,294.73 118.861,298.209 119.636,301.533 120.416,304.681 121.198,307.635 121.984,310.375 
 122.773,312.882 123.565,315.136 124.36,317.12 125.159,318.813 125.961,320.197 126.767,321.255 127.575,321.968 128.387,322.321 129.202,322.296 130.02,321.879 
 130.841,321.057 131.665,319.815 132.493,318.144 133.323,316.033 134.157,313.474 134.994,310.462 135.833,306.993 136.676,303.064 137.522,298.677 138.371,293.835 
 139.223,288.545 140.078,282.815 140.936,276.658 141.797,270.089 142.661,263.127 143.528,255.794 144.398,248.113 145.271,240.115 146.146,231.829 147.025,223.291 
 147.906,214.538 148.791,205.608 149.678,196.546 150.568,187.394 151.46,178.199 152.356,169.01 153.254,159.873 154.156,150.839 155.059,141.956 155.966,133.274 
 156.875,124.842 157.787,116.705 158.702,108.909 159.62,